Run block to Install pandas 

In [ ]:
%pip install pandas

How to clean data from https://sapit-iex-prod-lizard-customer-influence-workbench-ciw-6f02fb4d.cfapps.eu10-004.hana.ondemand.com/cccissessionowner/index.html#/DetailView/293,I311255

Change CSV path and run 

In [ ]:
import pandas as pd

def clean_data(df):
    # Drop column: 'FBA Comments'
    df = df.drop(columns=['FBA Comments'])
    # Drop columns: 'FBA Status', 'User ID', 'Internal Project ID'
    df = df.drop(columns=['FBA Status', 'User ID', 'Internal Project ID'])
    # Drop columns: 'Is Submitter ?', 'ERP Customer No.' and 6 other columns
    df = df.drop(columns=['Is Submitter ?', 'ERP Customer No.', 'Account Type', 'Voter Zip Code', 'Voter City', 'Voter Street', 'Voter Email', 'Voter Name'])
    # Drop column: 'INM Project Name'
    df = df.drop(columns=['INM Project Name'])
    # Drop columns: 'Coach Name', 'Category'
    df = df.drop(columns=['Coach Name', 'Category'])
    # Drop columns: 'IR Link', 'IR Title'
    df = df.drop(columns=['IR Link', 'IR Title'])
    # Drop columns: 'Decision Reason', 'Reference'
    df = df.drop(columns=['Decision Reason', 'Reference'])
    # Drop column: 'Views'
    df = df.drop(columns=['Views'])
    # Drop column: 'Customer Votes'
    df = df.drop(columns=['Customer Votes'])
    # Drop column: 'Last Status Change Date'
    df = df.drop(columns=['Last Status Change Date'])
    # Drop column: 'Status'
    df = df.drop(columns=['Status'])
    # Drop column: 'Reason Comments'
    df = df.drop(columns=['Reason Comments'])
    # Change column type to datetime64[ns] for column: 'Voted Date'
    df = df.astype({'Voted Date': 'datetime64[ns]'})
    # Change column type to string for column: 'Voter Company'
    df = df.astype({'Voter Company': 'string'})
    # Drop duplicate rows in columns: 'IR ID', 'Voter Company'
    df = df.drop_duplicates(subset=['IR ID', 'Voter Company'])
    # Change column type to category for column: 'Voter Company'
    df = df.astype({'Voter Company': 'category'})
    return df

# Loaded variable 'df' from URI: 
df = pd.read_csv(r'c:\\Users\\I586445\\OneDrive - National University of Ireland, Galway\\Documents\\InfluenceScripts\\Copy of Improvements_and_Votes(18-06-2024, 13_35).csv')

df_clean = clean_data(df.copy())
df_clean.head()

How to create consumer Foreign keys to connect Jira to influence 

In [ ]:
import csv
import re
import os

# Specify the path to your CSV file
csv_file_path = r"C:\Users\I586445\Downloads\17-06 Model Update\FPA34 17_06.csv"
# Construct the output file path
output_folder = os.path.dirname(csv_file_path)
output_file_path = os.path.join(output_folder, "Consumer Foreign keys.csv")

# Open the CSV file for reading and the output file for writing
with open(csv_file_path, 'r', encoding='utf-8-sig') as file, open(output_file_path, 'w', newline='', encoding='utf-8') as output_file:
    # Create a CSV reader object
    csv_reader = csv.reader(file)
    # Create a CSV writer object
    csv_writer = csv.writer(output_file)
    
    # Write the header row with selected columns
    csv_writer.writerow(["ISSUEKEY", "CONSUMER"])
    
    # Process the header row to get the column indices
    header = next(csv_reader)
    column_indices = {}
    for column_name in ["ISSUEKEY", "CONSUMER"]:
        try:
            column_indices[column_name] = header.index(column_name)
        except ValueError:
            # Handle the case if any of the required columns are not found in the header
            print(f"Column '{column_name}' not found in the CSV file.")
            exit(1)

    # Process each row and write selected columns to the output file
    for row in csv_reader:
        issue_key = row[column_indices["ISSUEKEY"]]
        consumer_digits = row[column_indices["CONSUMER"]]
        
    
        # Extract the "Consumer Digits" from the "Custom field (Consumer)" column using regex
        matches_consumer = re.findall(r"\bhttps://influence.sap.com.*?(\d{6})", consumer_digits)
        
        # If matches are found, create a new row for each match
        if matches_consumer:
            for match in matches_consumer:
                # Create a new row with details from the original row and the corresponding match
                new_row = [issue_key, match]
                csv_writer.writerow(new_row)
        else:
            # If no matches found, create a new row with details from the original row and an empty "Consumer Digits"
            new_row = [issue_key, ""]
            csv_writer.writerow(new_row)

print("Code executed successfully.")
